In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd


In [4]:
usda_tract = gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054 2/USDA_Rural_Housing_by_Tract.shp")
usda_tract = usda_tract.to_crs(epsg=4326)

In [22]:
tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")

In [23]:
tracts["is_rural"] = tracts["GEOID"].isin(usda_tract["GEOID"].astype(str))

In [24]:
# Assign pool using logic from QAP
atlanta_fips = {"089", "067", "121", "135"}

def assign_pool(row):
    if row["is_rural"]:
        return "rural"
    elif row["COUNTYFP"] in atlanta_fips:
        return "atlanta metro"
    else:
        return "other metro"

tracts["pool"] = tracts.apply(assign_pool, axis=1)

In [ ]:
def excel_sheets_to_dataframes(excel_file_path):

    # Load the Excel file
    excel_data = pd.ExcelFile(excel_file_path)
    
    dataframes = {}

    # Iterate over each sheet
    for sheet_name in excel_data.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

        # # Optionally drop rows and columns that are completely empty
        df.dropna(how='all', axis=0, inplace=True)
        df.dropna(how='all', axis=1, inplace=True)

        sanitized_name = sheet_name.lower().replace(" ", "_")
        print(sanitized_name)

        # Store the DataFrame in the dictionary
        dataframes[sanitized_name] = df

        # Save them as csvs
        df.to_csv("../../data/preprocessed/scoring_indicators/stable_communities/"+sanitized_name+"_2024.csv")
        print("../../data/preprocessed/scoring_indicators/stable_communities/"+sanitized_name+"_2024.csv")
    return dataframes


In [ ]:
excel_file = "../../data/raw/scoring_indicators/stable_communities/2024stablecommunities.xlsx"
dfs_dict = excel_sheets_to_dataframes(excel_file)

In [27]:
print(dfs_dict['environmental_health_index']['2020 Census Tract'].nunique())
print(dfs_dict['transit_access_index']['2020 Census Tract'].nunique())
print(dfs_dict['above_poverty_level']['2020 Census Tract'].nunique())
print(dfs_dict['median_income']['2020 Census Tract'].nunique())
print(dfs_dict['jobs_proximity_index']['2020 Census Tract'].nunique())

2796
2796
2796
2796
2796


In [51]:
# Merge the first two
df_merged = pd.merge(dfs_dict['environmental_health_index'], 
                     dfs_dict['transit_access_index'],
                     on="2020 Census Tract",
                     how="outer")

# Merge the next
df_merged = pd.merge(df_merged, 
                     dfs_dict['above_poverty_level'], 
                     on="2020 Census Tract", 
                     how="outer")

# Merge the next
df_merged = pd.merge(df_merged, 
                     dfs_dict['median_income'], 
                     on="2020 Census Tract", 
                     how="outer")

# Merge the final
df_merged = pd.merge(df_merged, 
                     dfs_dict['jobs_proximity_index'], 
                     on="2020 Census Tract", 
                     how="outer")

In [52]:
df_merged.dtypes

2020 Census Tract                                  int64
Environmental Health Index                       float64
Transit Access Index                              object
Percent of Population Above the Poverty Level     object
Median Income                                     object
Jobs Proximity Index                             float64
dtype: object

In [53]:
df_merged["2020 Census Tract"] = df_merged["2020 Census Tract"].astype(str).str.strip()
tracts["GEOID"] = tracts["GEOID"].astype(str).str.strip()

# Merge in pool info based on GEOID
df_merged = pd.merge(
    df_merged,
    tracts[["GEOID", "pool"]],
    left_on="2020 Census Tract",
    right_on="GEOID",
    how="left"
)

In [54]:
# Values from Stable Communities excel on DCA website
pool_medians = {
    "atlanta metro": {
        "Environmental Health Index": 0.286,
        "Transit Access Index": 0.600,
        "Percent of Population Above the Poverty Level": 91.2,
        "Median Income": 78538,
        "Jobs Proximity Index": 0.632
    },
    "other metro": {
        "Environmental Health Index": 0.430,
        "Transit Access Index": 0.609,
        "Percent of Population Above the Poverty Level": 86.4,
        "Median Income": 60044,
        "Jobs Proximity Index": 0.597
    },
    "rural": {
        "Environmental Health Index": 0.672,
        "Transit Access Index": 0.538,
        "Percent of Population Above the Poverty Level": 85.4,
        "Median Income": 55096,
        "Jobs Proximity Index": 0.577
    }
}

In [55]:
indicator_columns = list(pool_medians["rural"].keys()) 

for col in indicator_columns:
    df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

    new_col = f"above_median_{col}"
    df_merged[new_col] = 0  

    for pool_name in pool_medians:
        threshold = pool_medians[pool_name][col]
        mask = df_merged["pool"].str.lower() == pool_name 
        df_merged.loc[mask, new_col] = np.where(df_merged.loc[mask, col] > threshold, 1, 0)

In [56]:
if not df_merged.empty:
    df_merged.to_csv('../../data/processed/scoring_indicators/stable_communities_2024_processed_v3.csv', index=False)
else:
    print("Warning: df_merged is empty. CSV file not saved.")